# Détection automatique d'erreurs d'OCR

## Imports

In [1]:
import nltk
from thefuzz import fuzz
from thefuzz import process
from Levenshtein import distance

Nous allons d'abord utiliser la distance de Levenshtein, selon sa formule originale (1965)

In [2]:
distance("bulletin", "ballotins")

3

Nous allons ensuite faire appel à la librairie TheFuzz (anciennement fuzzywuzzy) qui repose sur python-Levenshtein (https://github.com/seatgeek/thefuzz).
Celle-ci permet de normaliser le résultat en calculant un ratio (pourcentage) de similarité entre deux chaînes de caractères :

In [3]:
fuzz.ratio("bulletin", "ballotins")

71

La fonction  `partial_ratio` se focalise sur la sous-chaîne (substring) la plus similaire : 

In [4]:
fuzz.partial_ratio("Grand", "Grand-Place")

100

La fonction `token_sort_ratio` ignore l'ordre des mots :

In [5]:
fuzz.token_sort_ratio("archives de la ville de Bruxelles", "de Bruxelles la ville de archives")

100

La fonction `token_set_ratio` ignore les répétitions de mots :

In [6]:
fuzz.token_set_ratio("plus jamais ça !", "plus jamais Jamais JAMAIS ça !!!")

100

Chargeons à présent le fichier `all.txt` (tronqué) : 

In [7]:
data_path = "../data/all.txt"
LIMIT = 50000000
text = open(data_path, "r", encoding="utf-8").read()[:LIMIT]

tokens = nltk.wordpunct_tokenize(text)
print(f"{len(tokens)} words (tokens) found")

vocab = set(tokens)
print(f"{len(vocab)} different word forms")

476914 words (tokens) found
70400 different word forms


Nous pouvons identifier les variantes d'un mot par ordre décroissant de similarité : 

In [8]:
word = "bruxelles"
process.extract(word, vocab, limit=15)

[('BRUXELLES', 100),
 ('Bruxellesà', 100),
 ('Bruxelles', 100),
 ('Bruxelle', 94),
 ('xelles', 90),
 ('lle', 90),
 ('EL', 90),
 ('LL', 90),
 ('lée', 90),
 ('uX', 90),
 ('BRUXELLESPelisses', 90),
 ('ru', 90),
 ('îles', 90),
 ('Ru', 90),
 ('BRU', 90)]

Les caractères accentués ne sont pas pris en compte pour le calcul du score, mais cela reste une bonne base pour récupérer les occurrences mal OCRisées d'un mot.